In [4]:
import torch
from loss import VaeLoss, BetaVaeLoss, AdaGVaeLoss
from main import make_datasets, train_loop, display_generated_grid, make_system
from model import VAE
import torch_optimizer as optim_extra
import torch
import numpy as np

In [ ]:
vae = train_loop(*make_system('ada-gvae', 'radam'), 3)
display_generated_grid(vae)

In [ ]:
vae = train_loop(*make_system('vae', 'radam'), 4)
display_generated_grid(vae)

In [ ]:
vae = train_loop(*make_system('beta-vae', 'radam'), 4)
display_generated_grid(vae)